# Collecting Historical Currency Exchange Rates

## Background

If you have any interest in collecting data about historical exchange rates, you may have accessed some websites that can offer this type of data. For example, [oanda.com](https://www.oanda.com/currency-converter/en/?from=CAD&to=USD&amount=1) is one that can be used to perform this task, however there is no API available (only for institutional clients, and probably paid access only). There are a host of others that offer paid access, but one that is relatively easy to use and offers free access (limited to 5,000 requests per month) is [currencyscoop.com](https://currencyscoop.com/api-documentation).

Note that the free access does not provide the ability to query a timeseries - you can query individual dates historically but each will only return one day of values. Timeseries queries are only available at the membership level that at the time of this writing cost $99 per month. Still, we can build our own timeseries if we don't mind making 5,000 request maximum per month. Another restriction on the free account is that the base currency cannot be changed from USD, although you can always do some conversion from one currency to another to form your base currency as a workaround.

You will probably need to know how to specify the currencies for your particular solution, and all of the currencies supported are listed on [https://currencyscoop.com/supported-currencies](https://currencyscoop.com/supported-currencies)

## Requirements

You must request an API key from [currencyscoop.com](https://currencyscoop.com/signup). The process is straightforward - simply your name, and email address and a password are required. Once you create your account, your API key will be displayed on the Account Details section of the page. This is the only requirements you need to create the request.

For this exercise, we will be using the historical endpoint which has the URL in the format:

https://api.currencyscoop.com/v1/historical?api_key=xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx&base=USD&date=YYYY-MM-DD&symbols=CR1,CR2,CR3

## Solution

In [1]:
# Enter your specific API Key into the constant value below

API_KEY = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"

In [2]:
# Import all the necessary libraries we will use for this exercise

from datetime import datetime
import pandas as pd
import numpy as np
import requests
import json

In [3]:
# For example, I am interested in retrieving exchange rates agains the USD for the
# Brazilian Real, and the Canadian Dollar, so I enter their symbols in the list

CUR_SYM = [
      "BRL"
    , "CAD"
]

In [4]:
# During the execution of the code, we will need a string representation of the list

csv_cur_sym = ','.join(CUR_SYM)

In [5]:
# Suppose I am interested in all exchange rates for the currencies I chose
# starting at Jan. 1, 2019 - let's set up the start date here

dx = datetime(2019, 1, 1).date()
dx

datetime.date(2019, 1, 1)

In [6]:
# Let's create a pandas DataFrame with business days only as the values for the first column

#df_currencyscoop = pd.bdate_range(start = dx, end = datetime.today().date()).to_frame(index=False, name="Date")
# Use the line above to have a numeric index - we actually have a useful reason to use a date index, so...

df_currencyscoop = pd.bdate_range(start = dx, end = datetime.today().date()).to_frame(index=True, name="Date")
df_currencyscoop.head(10)

,Date
2019-01-01,2019-01-01
2019-01-02,2019-01-02
2019-01-03,2019-01-03
2019-01-04,2019-01-04
2019-01-07,2019-01-07
2019-01-08,2019-01-08
2019-01-09,2019-01-09
2019-01-10,2019-01-10
2019-01-11,2019-01-11
2019-01-14,2019-01-14


In [7]:
# Next we define a function that can be called by pandas apply() to create another column 

def construct_currencyscoop_url(trade_date):
    return f"https://api.currencyscoop.com/v1/historical?api_key={API_KEY}&base=USD&date={trade_date.strftime('%Y-%m-%d')}&symbols={csv_cur_sym}"


In [8]:
# Now we use that function to generate the URL column in our DataFrame

df_currencyscoop["URL"] = df_currencyscoop["Date"].apply(lambda x: construct_currencyscoop_url(x))
df_currencyscoop.head(10)

,Date,URL
2019-01-01,2019-01-01,https://api.currencyscoop.com/v1/historical?ap...
2019-01-02,2019-01-02,https://api.currencyscoop.com/v1/historical?ap...
2019-01-03,2019-01-03,https://api.currencyscoop.com/v1/historical?ap...
2019-01-04,2019-01-04,https://api.currencyscoop.com/v1/historical?ap...
2019-01-07,2019-01-07,https://api.currencyscoop.com/v1/historical?ap...
2019-01-08,2019-01-08,https://api.currencyscoop.com/v1/historical?ap...
2019-01-09,2019-01-09,https://api.currencyscoop.com/v1/historical?ap...
2019-01-10,2019-01-10,https://api.currencyscoop.com/v1/historical?ap...
2019-01-11,2019-01-11,https://api.currencyscoop.com/v1/historical?ap...
2019-01-14,2019-01-14,https://api.currencyscoop.com/v1/historical?ap...


In [9]:
# Let's add a column for each of the currencies in our list

df_currencyscoop = df_currencyscoop.reindex(columns=list(df_currencyscoop.columns) + CUR_SYM, fill_value=0.0)
df_currencyscoop.head(10)

,Date,URL,BRL,CAD
2019-01-01,2019-01-01,https://api.currencyscoop.com/v1/historical?ap...,0.0,0.0
2019-01-02,2019-01-02,https://api.currencyscoop.com/v1/historical?ap...,0.0,0.0
2019-01-03,2019-01-03,https://api.currencyscoop.com/v1/historical?ap...,0.0,0.0
2019-01-04,2019-01-04,https://api.currencyscoop.com/v1/historical?ap...,0.0,0.0
2019-01-07,2019-01-07,https://api.currencyscoop.com/v1/historical?ap...,0.0,0.0
2019-01-08,2019-01-08,https://api.currencyscoop.com/v1/historical?ap...,0.0,0.0
2019-01-09,2019-01-09,https://api.currencyscoop.com/v1/historical?ap...,0.0,0.0
2019-01-10,2019-01-10,https://api.currencyscoop.com/v1/historical?ap...,0.0,0.0
2019-01-11,2019-01-11,https://api.currencyscoop.com/v1/historical?ap...,0.0,0.0
2019-01-14,2019-01-14,https://api.currencyscoop.com/v1/historical?ap...,0.0,0.0


In [10]:
# Next we iterate through each of the rows in the DataFrame, using the URL that was generated
for r in df_currencyscoop.index.to_list(): # We can limit the number of requests by appending a list slice such as [:10]
    url = df_currencyscoop.loc[r]["URL"]
    print(url)
    resp = requests.get(url)
    if resp.status_code == 200:
        resp_text = resp.text
        resp_json = json.loads(resp_text)
        for c in resp_json["response"]["rates"].keys():
            v = resp_json["response"]["rates"][c]
            df_currencyscoop.loc[r, c] = v

https://api.currencyscoop.com/v1/historical?api_key=xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx&base=USD&date=2019-01-01&symbols=BRL,CAD
https://api.currencyscoop.com/v1/historical?api_key=xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx&base=USD&date=2019-01-02&symbols=BRL,CAD
https://api.currencyscoop.com/v1/historical?api_key=xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx&base=USD&date=2019-01-03&symbols=BRL,CAD
https://api.currencyscoop.com/v1/historical?api_key=xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx&base=USD&date=2019-01-04&symbols=BRL,CAD
https://api.currencyscoop.com/v1/historical?api_key=xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx&base=USD&date=2019-01-07&symbols=BRL,CAD
https://api.currencyscoop.com/v1/historical?api_key=xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx&base=USD&date=2019-01-08&symbols=BRL,CAD
https://api.currencyscoop.com/v1/historical?api_key=xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx&base=USD&date=2019-01-09&symbols=BRL,CAD
https://api.currencyscoop.com/v1/historical?api_key=xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx&base=USD&date=2019-01-10&symbols=

In [11]:
# Let's take a peak at what currencyscoop returned at the head

df_currencyscoop.head(10)

,Date,URL,BRL,CAD
2019-01-01,2019-01-01,https://api.currencyscoop.com/v1/historical?ap...,3.881175,1.347588
2019-01-02,2019-01-02,https://api.currencyscoop.com/v1/historical?ap...,3.811706,1.358658
2019-01-03,2019-01-03,https://api.currencyscoop.com/v1/historical?ap...,3.744075,1.348520
2019-01-04,2019-01-04,https://api.currencyscoop.com/v1/historical?ap...,3.714286,1.339817
2019-01-07,2019-01-07,https://api.currencyscoop.com/v1/historical?ap...,3.714667,1.329348
2019-01-08,2019-01-08,https://api.currencyscoop.com/v1/historical?ap...,3.722128,1.331395
2019-01-09,2019-01-09,https://api.currencyscoop.com/v1/historical?ap...,3.682475,1.321223
2019-01-10,2019-01-10,https://api.currencyscoop.com/v1/historical?ap...,3.690364,1.323251
2019-01-11,2019-01-11,https://api.currencyscoop.com/v1/historical?ap...,3.708313,1.325484
2019-01-14,2019-01-14,https://api.currencyscoop.com/v1/historical?ap...,3.699844,1.327843


In [12]:
# Let's take a peak at what currencyscoop returned at the tail

df_currencyscoop.tail(10)

,Date,URL,BRL,CAD
2022-07-11,2022-07-11,https://api.currencyscoop.com/v1/historical?ap...,5.378371,1.301992
2022-07-12,2022-07-12,https://api.currencyscoop.com/v1/historical?ap...,5.436768,1.300745
2022-07-13,2022-07-13,https://api.currencyscoop.com/v1/historical?ap...,5.393405,1.297552
2022-07-14,2022-07-14,https://api.currencyscoop.com/v1/historical?ap...,5.424461,1.310974
2022-07-15,2022-07-15,https://api.currencyscoop.com/v1/historical?ap...,5.407798,1.303204
2022-07-18,2022-07-18,https://api.currencyscoop.com/v1/historical?ap...,5.438638,1.297232
2022-07-19,2022-07-19,https://api.currencyscoop.com/v1/historical?ap...,5.414491,1.285727
2022-07-20,2022-07-20,https://api.currencyscoop.com/v1/historical?ap...,5.471833,1.287450
2022-07-21,2022-07-21,https://api.currencyscoop.com/v1/historical?ap...,5.498915,1.288975
2022-07-22,2022-07-22,https://api.currencyscoop.com/v1/historical?ap...,5.497669,1.291508


In [13]:
# In fact, I am interested in the inverse exchange rate i.e.
# how much of currency X do I need in order to buy 1 USD?
# So we can apply the inverse math operation to our DataFrame
# and name the resulting columns with a "_INV" suffix
# Any division by zero errors (resulting in an inf) will be converted to 0

df_currencyscoop = df_currencyscoop.join(1/df_currencyscoop[CUR_SYM].add_suffix("_INV")).replace([np.inf, -np.inf], 0)
df_currencyscoop.head(10)

,Date,URL,BRL,CAD,BRL_INV,CAD_INV
2019-01-01,2019-01-01,https://api.currencyscoop.com/v1/historical?ap...,3.881175,1.347588,0.257654,0.742067
2019-01-02,2019-01-02,https://api.currencyscoop.com/v1/historical?ap...,3.811706,1.358658,0.262350,0.736020
2019-01-03,2019-01-03,https://api.currencyscoop.com/v1/historical?ap...,3.744075,1.348520,0.267089,0.741554
2019-01-04,2019-01-04,https://api.currencyscoop.com/v1/historical?ap...,3.714286,1.339817,0.269231,0.746371
2019-01-07,2019-01-07,https://api.currencyscoop.com/v1/historical?ap...,3.714667,1.329348,0.269203,0.752249
2019-01-08,2019-01-08,https://api.currencyscoop.com/v1/historical?ap...,3.722128,1.331395,0.268664,0.751092
2019-01-09,2019-01-09,https://api.currencyscoop.com/v1/historical?ap...,3.682475,1.321223,0.271556,0.756875
2019-01-10,2019-01-10,https://api.currencyscoop.com/v1/historical?ap...,3.690364,1.323251,0.270976,0.755715
2019-01-11,2019-01-11,https://api.currencyscoop.com/v1/historical?ap...,3.708313,1.325484,0.269664,0.754442
2019-01-14,2019-01-14,https://api.currencyscoop.com/v1/historical?ap...,3.699844,1.327843,0.270282,0.753101


In [14]:
# Lastly, we typically do not execute currency trades at the official exchange rates.
# So we will create a new set of columns applying a discount of 1% and 2%,
# naming those with additional suffixes _1 and _2

# First the 1% discount
df_currencyscoop = df_currencyscoop.join(
    0.99*df_currencyscoop[('_INV,'.join(CUR_SYM) + '_INV').split(',')].add_suffix("_1")).replace([np.inf, -np.inf], 0)

# Next the 2% discount
df_currencyscoop = df_currencyscoop.join(
    0.98*df_currencyscoop[('_INV,'.join(CUR_SYM) + '_INV').split(',')].add_suffix("_2")).replace([np.inf, -np.inf], 0)

# Lastly, let's sort the currency columns in alphabetical name order left to right
df_currencyscoop = df_currencyscoop[df_currencyscoop.columns.to_list()[:2] + sorted(df_currencyscoop.columns.to_list()[2:])]

df_currencyscoop.head(10)

,Date,URL,BRL,BRL_INV,BRL_INV_1,BRL_INV_2,CAD,CAD_INV,CAD_INV_1,CAD_INV_2
2019-01-01,2019-01-01,https://api.currencyscoop.com/v1/historical?ap...,3.881175,0.257654,0.255077,0.252501,1.347588,0.742067,0.734646,0.727225
2019-01-02,2019-01-02,https://api.currencyscoop.com/v1/historical?ap...,3.811706,0.262350,0.259726,0.257103,1.358658,0.736020,0.728660,0.721300
2019-01-03,2019-01-03,https://api.currencyscoop.com/v1/historical?ap...,3.744075,0.267089,0.264418,0.261747,1.348520,0.741554,0.734138,0.726723
2019-01-04,2019-01-04,https://api.currencyscoop.com/v1/historical?ap...,3.714286,0.269231,0.266538,0.263846,1.339817,0.746371,0.738907,0.731443
2019-01-07,2019-01-07,https://api.currencyscoop.com/v1/historical?ap...,3.714667,0.269203,0.266511,0.263819,1.329348,0.752249,0.744726,0.737204
2019-01-08,2019-01-08,https://api.currencyscoop.com/v1/historical?ap...,3.722128,0.268664,0.265977,0.263290,1.331395,0.751092,0.743581,0.736070
2019-01-09,2019-01-09,https://api.currencyscoop.com/v1/historical?ap...,3.682475,0.271556,0.268841,0.266125,1.321223,0.756875,0.749306,0.741737
2019-01-10,2019-01-10,https://api.currencyscoop.com/v1/historical?ap...,3.690364,0.270976,0.268266,0.265556,1.323251,0.755715,0.748157,0.740600
2019-01-11,2019-01-11,https://api.currencyscoop.com/v1/historical?ap...,3.708313,0.269664,0.266968,0.264271,1.325484,0.754442,0.746897,0.739353
2019-01-14,2019-01-14,https://api.currencyscoop.com/v1/historical?ap...,3.699844,0.270282,0.267579,0.264876,1.327843,0.753101,0.745570,0.738039


In [15]:
# We can now export the DataFrame to an Excel file for further use
# including to be able to append other data to it from subsequent runs

day0 = df_currencyscoop.iloc[0]["Date"].strftime("%Y%m%d")
dayn = df_currencyscoop.iloc[-1]["Date"].strftime("%Y%m%d")

filename = f"currencyscoop_{day0}_{dayn}.xlsx"

df_currencyscoop.to_excel("currencyscoop.xlsx")

print(f"File {filename} created")

File currencyscoop_20190101_20220722.xlsx created


In [16]:
# Suppose we needed to have CAD as the base currency
# Much of the work has already been done - make a copy of the DataFrame

df_baseCAD = df_currencyscoop.copy()
df_baseCAD.head(10)

,Date,URL,BRL,BRL_INV,BRL_INV_1,BRL_INV_2,CAD,CAD_INV,CAD_INV_1,CAD_INV_2
2019-01-01,2019-01-01,https://api.currencyscoop.com/v1/historical?ap...,3.881175,0.257654,0.255077,0.252501,1.347588,0.742067,0.734646,0.727225
2019-01-02,2019-01-02,https://api.currencyscoop.com/v1/historical?ap...,3.811706,0.262350,0.259726,0.257103,1.358658,0.736020,0.728660,0.721300
2019-01-03,2019-01-03,https://api.currencyscoop.com/v1/historical?ap...,3.744075,0.267089,0.264418,0.261747,1.348520,0.741554,0.734138,0.726723
2019-01-04,2019-01-04,https://api.currencyscoop.com/v1/historical?ap...,3.714286,0.269231,0.266538,0.263846,1.339817,0.746371,0.738907,0.731443
2019-01-07,2019-01-07,https://api.currencyscoop.com/v1/historical?ap...,3.714667,0.269203,0.266511,0.263819,1.329348,0.752249,0.744726,0.737204
2019-01-08,2019-01-08,https://api.currencyscoop.com/v1/historical?ap...,3.722128,0.268664,0.265977,0.263290,1.331395,0.751092,0.743581,0.736070
2019-01-09,2019-01-09,https://api.currencyscoop.com/v1/historical?ap...,3.682475,0.271556,0.268841,0.266125,1.321223,0.756875,0.749306,0.741737
2019-01-10,2019-01-10,https://api.currencyscoop.com/v1/historical?ap...,3.690364,0.270976,0.268266,0.265556,1.323251,0.755715,0.748157,0.740600
2019-01-11,2019-01-11,https://api.currencyscoop.com/v1/historical?ap...,3.708313,0.269664,0.266968,0.264271,1.325484,0.754442,0.746897,0.739353
2019-01-14,2019-01-14,https://api.currencyscoop.com/v1/historical?ap...,3.699844,0.270282,0.267579,0.264876,1.327843,0.753101,0.745570,0.738039


In [17]:
# The column CAD_INV already contains the number of USD we can buy with 1 CAD
# So we can keep that column, and also use that number as a multiplier for BRL

(df_baseCAD["CAD_INV"] * df_baseCAD["BRL"]).head(10)

2019-01-01    2.880091
2019-01-02    2.805494
2019-01-03    2.776433
2019-01-04    2.772234
2019-01-07    2.794353
2019-01-08    2.795659
2019-01-09    2.787172
2019-01-10    2.788862
2019-01-11    2.797706
2019-01-14    2.786357
Freq: B, dtype: float64

In [18]:
# Whatever our list of original target currencies contains
# We want to keep all except CAD, and add USD
CUR_SYM_CAD = list(set(x for x in CUR_SYM + ["USD"] if x != "CAD"))
CUR_SYM_CAD

['USD', 'BRL']

In [19]:
# Regardless of how many currencies we have in our original list CUR_SYM
# the following will drop all columns we no longer need for the CAD base DataFrame

drop_cols = [x for x in list(df_baseCAD.columns) if x not in ["Date", "URL", "CAD_INV"] and ("_INV" in x or x == "CAD")]
drop_cols

['BRL_INV', 'BRL_INV_1', 'BRL_INV_2', 'CAD', 'CAD_INV_1', 'CAD_INV_2']

In [20]:
df_baseCAD.drop(columns=drop_cols, inplace=True)
df_baseCAD.head(10)

,Date,URL,BRL,CAD_INV
2019-01-01,2019-01-01,https://api.currencyscoop.com/v1/historical?ap...,3.881175,0.742067
2019-01-02,2019-01-02,https://api.currencyscoop.com/v1/historical?ap...,3.811706,0.736020
2019-01-03,2019-01-03,https://api.currencyscoop.com/v1/historical?ap...,3.744075,0.741554
2019-01-04,2019-01-04,https://api.currencyscoop.com/v1/historical?ap...,3.714286,0.746371
2019-01-07,2019-01-07,https://api.currencyscoop.com/v1/historical?ap...,3.714667,0.752249
2019-01-08,2019-01-08,https://api.currencyscoop.com/v1/historical?ap...,3.722128,0.751092
2019-01-09,2019-01-09,https://api.currencyscoop.com/v1/historical?ap...,3.682475,0.756875
2019-01-10,2019-01-10,https://api.currencyscoop.com/v1/historical?ap...,3.690364,0.755715
2019-01-11,2019-01-11,https://api.currencyscoop.com/v1/historical?ap...,3.708313,0.754442
2019-01-14,2019-01-14,https://api.currencyscoop.com/v1/historical?ap...,3.699844,0.753101


In [21]:
# Now all that's left to do is to rename our inverse CAD column

df_baseCAD.rename(columns={"CAD_INV": "USD"}, inplace=True)
df_baseCAD.head(10)

,Date,URL,BRL,USD
2019-01-01,2019-01-01,https://api.currencyscoop.com/v1/historical?ap...,3.881175,0.742067
2019-01-02,2019-01-02,https://api.currencyscoop.com/v1/historical?ap...,3.811706,0.736020
2019-01-03,2019-01-03,https://api.currencyscoop.com/v1/historical?ap...,3.744075,0.741554
2019-01-04,2019-01-04,https://api.currencyscoop.com/v1/historical?ap...,3.714286,0.746371
2019-01-07,2019-01-07,https://api.currencyscoop.com/v1/historical?ap...,3.714667,0.752249
2019-01-08,2019-01-08,https://api.currencyscoop.com/v1/historical?ap...,3.722128,0.751092
2019-01-09,2019-01-09,https://api.currencyscoop.com/v1/historical?ap...,3.682475,0.756875
2019-01-10,2019-01-10,https://api.currencyscoop.com/v1/historical?ap...,3.690364,0.755715
2019-01-11,2019-01-11,https://api.currencyscoop.com/v1/historical?ap...,3.708313,0.754442
2019-01-14,2019-01-14,https://api.currencyscoop.com/v1/historical?ap...,3.699844,0.753101


In [22]:
# Multiply all of our other exchanges rates (non-USD) by the CAD_INV (now called USD)

for c in CUR_SYM:
    if c != "CAD":
        df_baseCAD[c] = (df_baseCAD["USD"] * df_baseCAD[c])

df_baseCAD.head(10)

,Date,URL,BRL,USD
2019-01-01,2019-01-01,https://api.currencyscoop.com/v1/historical?ap...,2.880091,0.742067
2019-01-02,2019-01-02,https://api.currencyscoop.com/v1/historical?ap...,2.805494,0.736020
2019-01-03,2019-01-03,https://api.currencyscoop.com/v1/historical?ap...,2.776433,0.741554
2019-01-04,2019-01-04,https://api.currencyscoop.com/v1/historical?ap...,2.772234,0.746371
2019-01-07,2019-01-07,https://api.currencyscoop.com/v1/historical?ap...,2.794353,0.752249
2019-01-08,2019-01-08,https://api.currencyscoop.com/v1/historical?ap...,2.795659,0.751092
2019-01-09,2019-01-09,https://api.currencyscoop.com/v1/historical?ap...,2.787172,0.756875
2019-01-10,2019-01-10,https://api.currencyscoop.com/v1/historical?ap...,2.788862,0.755715
2019-01-11,2019-01-11,https://api.currencyscoop.com/v1/historical?ap...,2.797706,0.754442
2019-01-14,2019-01-14,https://api.currencyscoop.com/v1/historical?ap...,2.786357,0.753101


## Done!